In [1]:
# drive mount
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Import Library

In [3]:
!pip install ttach
!pip install timm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import random
import pandas as pd
import numpy as np
import os
import cv2

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import timm

from tqdm.auto import tqdm
from copy import deepcopy

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import ttach as tta

import torchvision.models as models

from sklearn.metrics import f1_score

import random
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
# device 할당
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [4]:
CFG = {
    'IMG_SIZE' : 224,
    'IMG_SIZE_H': 220,
    'IMG_SIZE_W': 275,
    'EPOCHS': 100,
    'LEARNING_RATE': 3e-4,
    #'LEARNING_RATE': 1e-4,
    'BATCH_SIZE': 64,
    'SEED': 41,
    'PATIENCE' : 5,
    'group_num' : 6
}
# 6 5 4 3

In [5]:
# RandomSeed
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED'])

## Data Preprocessing

In [6]:
def class_grouping(df, group_num):
  tmp_df = pd.DataFrame(df.iloc[:,2:].value_counts())
  tmp_df.reset_index(inplace=True)
  tmp_df['Group'] = -1

  name_list = []

  name_list.append(list(train.iloc[:,2:].value_counts()[:1].index))
  name_list.append(list(train.iloc[:,2:].value_counts()[1:3].index))
  name_list.append(list(train.iloc[:,2:].value_counts()[3:13].index))
  name_list.append(list(train.iloc[:,2:].value_counts()[13:23].index))
  name_list.append(list(train.iloc[:,2:].value_counts()[23:33].index))
  name_list.append(list(train.iloc[:,2:].value_counts()[33:-10].index))
  name_list.append(list(train.iloc[:,2:].value_counts()[-10:].index))


  for name in  name_list[group_num]:
    tmp_df.loc[tmp_df['artist'] == name[0],'Group'] = group_num

  tmp_df.drop(0,axis=1,inplace=True)

  return pd.merge(train,tmp_df).sort_values('id').set_index('id')

########################################################################################

def train_sampling(df, group_num):
  test_df = df.copy()

  test_df.loc[test_df['Group'] != group_num,'artist'] = 50
  
  le_tmp = preprocessing.LabelEncoder()
  test_df['artist'] = le_tmp.fit_transform(test_df['artist'].values)
  cnt = len(le_tmp.classes_)


  sample_cnt = int(np.max(test_df['artist'].value_counts().values[1:]))
  print('sample_cnt :', sample_cnt)
  df_list = [test_df[test_df['artist'] == cnt-1].sample(sample_cnt)]

  for i in range(cnt-1):
    df_list.append(test_df[test_df['artist'] == i])

  return pd.concat(df_list), le_tmp

########################################################################################

def img_path_change(img_path):
  return '/content/drive/MyDrive/Colab Notebooks/Artist_classification' + str(img_path)[1:]



In [7]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Artist_classification/data/train.csv')

train.loc[3986] = [3986,'./train/3986.jpg','Alfred Sisley']
train.loc[3896,'artist'] = 'Titian'

train['img_path'] = train['img_path'].apply(img_path_change)


train_grouping = class_grouping(train,CFG['group_num'])

le = preprocessing.LabelEncoder()
train_grouping['artist'] = le.fit_transform(train_grouping['artist'].values)

In [8]:
train_sampled,le_group = train_sampling(train_grouping,CFG['group_num'])
train_sampled.reset_index(inplace=True,drop=True,)

sample_cnt : 50


In [38]:
train_sampled

,img_path,artist,Group
0,/content/drive/MyDrive/Colab Notebooks/Artist_...,7,-1
1,/content/drive/MyDrive/Colab Notebooks/Artist_...,7,-1
2,/content/drive/MyDrive/Colab Notebooks/Artist_...,7,-1
3,/content/drive/MyDrive/Colab Notebooks/Artist_...,7,-1
4,/content/drive/MyDrive/Colab Notebooks/Artist_...,7,-1
...,...,...,...
476,/content/drive/MyDrive/Colab Notebooks/Artist_...,6,5
477,/content/drive/MyDrive/Colab Notebooks/Artist_...,6,5
478,/content/drive/MyDrive/Colab Notebooks/Artist_...,6,5
479,/content/drive/MyDrive/Colab Notebooks/Artist_...,6,5


## 원본

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Artist_classification/data/train.csv')
train.shape

(5911, 3)

In [ ]:
train.loc[3986] = [3986,'./train/3986.jpg','Alfred Sisley']
train.loc[3896,'artist'] = 'Titian'

In [ ]:
def img_path_change(img_path):
  return '/content/drive/MyDrive/Colab Notebooks/Artist_classification' + str(img_path)[1:]

train['img_path'] = train['img_path'].apply(img_path_change)

In [ ]:
train['img_path'][0]

'/content/drive/MyDrive/Colab Notebooks/Artist_classification/train/0000.jpg'

In [ ]:
# Label Encoding : artist들을 범주형 데이터로 변환
# 화가 이름 50명
le = preprocessing.LabelEncoder()
train['artist_'] = le.fit_transform(train['artist'].values)

## Train / Validation Split

In [9]:
train_df, val_df, _, _ = train_test_split(train_sampled, train_sampled['artist'].values, test_size=0.2, random_state=CFG['SEED'],stratify=train_sampled['artist'].values)

#train_df, val_df, _, _ = train_test_split(train, train['artist'].values, test_size=0.2, random_state=CFG['SEED'],stratify=train['artist'].values)

In [42]:
#train_df = train_df.sort_values(by=['id'])
train_df.head()
print(train_df['Group'].value_counts())
print(train_df['artist'].value_counts())

 5    333
-1     51
Name: Group, dtype: int64
7    51
0    51
2    49
4    49
6    48
1    47
5    47
3    42
Name: artist, dtype: int64


In [ ]:
val_df = val_df.sort_values(by=['id'])
val_df.head()

## Data Load

In [10]:
# inference=True면 test 데이터라는 뜻.
# 따라서 target에 해당하는 artist를 return할 수 없음.
def get_data(df, infer=False):
  if infer:
    return df['img_path'].values
    
  return df['img_path'].values, df['artist'].values

In [11]:
# 파일 경로, 레이블
train_img_paths, train_labels = get_data(train_df)
val_img_paths, val_labels = get_data(val_df)

In [12]:
# 여기서 9등분하고 train_imgs_labels, val_imgs_labels 만들기
def split_image(paths,labels):
  img_list = []
  label_list = []
  himg_list = []
  #vimg_list = []

  for path, label in tqdm(zip(paths, labels),total=len(paths)):
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    height,width,c = image.shape

    half_h = height//2
    half_w = width//2
    queter_h = half_h//2
    queter_w = half_w//2

    pos_list = [
        [0,half_w, 0,half_h],
        [half_w,width, 0,half_h],
        [0,half_w, half_h,height],
        [half_w,width, half_h,height],
        [0,half_w, queter_h,height-queter_h],
        [half_w,width, queter_h,height-queter_h],
        [queter_w,width-queter_w, 0,half_h],
        [queter_w,width-queter_w, half_h,height],
        [queter_w,width-queter_w, queter_h,height-queter_h]
    ]

    if CFG['group_num'] >= 5 :
      for i in range(CFG['group_num'] * 5):
        rand_h = random.randrange(0, half_h)
        rand_w = random.randrange(0, half_w)

        img = cv2.resize(image[rand_h:rand_h+half_h,rand_w:rand_w+half_w], dsize=(CFG['IMG_SIZE_W'], CFG['IMG_SIZE_H']), interpolation=cv2.INTER_LINEAR)
        img_list.append(img)      

        histr = cv2.calcHist([img],[0],None,[256],[0,256]).reshape(-1)
        histg = cv2.calcHist([img],[1],None,[256],[0,256]).reshape(-1)
        histb = cv2.calcHist([img],[2],None,[256],[0,256]).reshape(-1)

        hist = np.concatenate((histr, histg,histb), axis=0)

        min_value = np.min(hist)
        max_value = np.max(hist)
        hist = (hist - min_value) / (max_value - min_value)

        himg_list.append(hist)

        label_list.append(label)
        
    
    else:
      for poses in pos_list:
        img = cv2.resize(image[poses[2]:poses[3],poses[0]:poses[1]], dsize=(CFG['IMG_SIZE_W'], CFG['IMG_SIZE_H']), interpolation=cv2.INTER_LINEAR)
        img_list.append(img)      

        histr = cv2.calcHist([img],[0],None,[256],[0,256]).reshape(-1)
        histg = cv2.calcHist([img],[1],None,[256],[0,256]).reshape(-1)
        histb = cv2.calcHist([img],[2],None,[256],[0,256]).reshape(-1)

        hist = np.concatenate((histr, histg,histb), axis=0)

        min_value = np.min(hist)
        max_value = np.max(hist)
        hist = (hist - min_value) / (max_value - min_value)

        himg_list.append(hist)

        label_list.append(label)

  return img_list, himg_list, label_list

In [13]:
train_imgs, train_himgs,  train_labels = split_image(train_img_paths, train_labels)

  0%|          | 0/324 [00:00<?, ?it/s]

In [14]:
val_imgs, val_himgs,  val_labels = split_image(val_img_paths, val_labels)

  0%|          | 0/82 [00:00<?, ?it/s]

## CustomDataset

In [15]:
# torch.utils.data.Dataset이라는 class를 상속받는 자식 클래스
class CustomDataset(Dataset):

  # 데이터셋을 처음 선언할 때, 자동으로 호출.
  # 몇 가지 인수들을 입력받도록 만들 수 있다.
  def __init__(self, imgs, himgs, labels, transforms=None):
    self.imgs = imgs
    self.himgs = himgs
    #self.vimgs = vimgs
    self.labels = labels
    self.transforms = transforms


  # 데이터셋에서 특정 1개의 샘플을 가져오기
  # index는 몇 번째 데이터를 가져올건지에 대한 변수.
  def __getitem__(self, index):
    image = self.imgs[index]
    himg = self.himgs[index]
    #vimg = self.vimgs[index]


    # 아래 dataset 선언을 보면 transform이 사용됨.
    if self.transforms is not None:
      image = self.transforms(image=image)['image']
      #himg = self.transforms(image=himg)['image']
      #vimg = self.transforms(image=vimg)['image']

 


    if self.labels is not None:
      label = self.labels[index]
      return image, himg, label
    else:
      return image, himg

  # 데이터셋의 길이 (총 샘플의 수)
  # 데이터셋을 선언하고 dataloader를 사용할 때 내부적으로 사용
  ## 데이터셋의 len을 알아야 데이터로더가 미니 배치를 사용할 수 있기 때문
  def __len__(self):
    return len(self.imgs)

In [16]:
class TestDataset(Dataset):
  def __init__(self, img_paths, labels, transforms=None):
    self.img_paths = img_paths
    self.labels = labels
    self.transforms = transforms


  def __getitem__(self, index):
    img_path = self.img_paths[index]

    image = cv2.imread(img_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, dsize=(CFG['IMG_SIZE_W'], CFG['IMG_SIZE_H']), interpolation=cv2.INTER_LINEAR)

    histr = cv2.calcHist([image],[0],None,[256],[0,256]).reshape(-1)
    histg = cv2.calcHist([image],[1],None,[256],[0,256]).reshape(-1)
    histb = cv2.calcHist([image],[2],None,[256],[0,256]).reshape(-1)

    hist = np.concatenate((histr, histg,histb), axis=0)

    min_value = np.min(hist)
    max_value = np.max(hist)
    hist = (hist - min_value) / (max_value - min_value)
    
    if self.transforms is not None:
      image = self.transforms(image=image)['image']
      #himage = self.transforms(image=himage)['image']
      #vimage = self.transforms(image=vimage)['image']



    if self.labels is not None:
      label = self.labels[index]
      return image, hist, label
    else:
      return image, hist,

  def __len__(self):
    return len(self.img_paths)

In [17]:
# Albumentation Augmentation
train_transform = A.Compose([
                            #A.Resize(CFG['IMG_SIZE_H'],CFG['IMG_SIZE_W']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            A.HorizontalFlip(p=0.5),
                            A.VerticalFlip(p=0.5),
                            ToTensorV2()
                            ])

test_transform = A.Compose([
                            #A.Resize(CFG['IMG_SIZE_H'],CFG['IMG_SIZE_W']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])


In [18]:
# Data Loader

train_dataset = CustomDataset(train_imgs, train_himgs, train_labels, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=2)
#train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], num_workers=2, sampler = sampler)

val_dataset = CustomDataset(val_imgs, val_himgs, val_labels, test_transform)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=2)

## Model Define

In [19]:
class Network_eff(nn.Module):
    #def __init__(self, num_classes=len(le.classes_)):
    def __init__(self, num_classes=len(le_group.classes_)):
        super(Network_eff, self).__init__()
        self.backbone = models.efficientnet_b0(pretrained=True) # b0 ~ b7

        self.fc1 = nn.Linear(768,512)
        self.fc2 = nn.Linear(512,500)
        
        self.classifier = nn.Linear(1500, num_classes)
        
    def forward(self, x1, x2): #x1 : 원본 이미지 , x2 : r256 + g256 + b256 = 768
        x1 = self.backbone(x1)
        
        x2 = F.relu(self.fc1(x2))
        x2 = F.relu(self.fc2(x2))

        x = torch.cat((x1,x2),1)

        x = self.classifier(x)
        return x



## Train

In [20]:
def train(model, optimizer, train_loader, test_loader, scheduler, device):
  # 모델을 device에 할당
  model.to(device)

  # early stopping
  es_count = 0

  # Loss 정의
  criterion = nn.CrossEntropyLoss().to(device)

  # Scheduler에서 사용할 변수 선언
  best_score = 0
  best_model = None

  for epoch in range(1, CFG['EPOCHS'] + 1):
    # model을 train 모드로 전환
    model.train()

    # loss값을 넣을 리스트 생성
    train_loss = []

    # Epoch 진행
    for img, himg, label in tqdm(iter(train_loader)):
      img, himg, label = img.float().to(device), himg.float().to(device), label.to(device)

      # 과거에 이용한 mini batch 내 이미지, 레이블을 바탕으로 계산된 Loss의 Gradient값이 optimizer에 할당되어 있는 것을 방지.
      optimizer.zero_grad()

      # pred값 
      model_pred = model(img, himg)

      # 선언한 Loss에 pred값과 정답을 넣기 
      loss = criterion(model_pred, label)

      # backpropagation
      loss.backward()

      # optimizer
      optimizer.step()

      # loss값 추가
      train_loss.append(loss.item())

    # 최종 loss값 생성
    tr_loss = np.mean(train_loss)

    val_loss, val_score = validation(model, criterion, test_loader, device)

    print(f'Epoch [{epoch}], Train Loss : [{tr_loss:.5f}] Val Loss : [{val_loss:.5f}] Val F1 Score : [{val_score:.5f}]')

    # Scheduler
    if scheduler is not None:
      scheduler.step()

    es_count += 1

    # val_score을 기준으로 best model 선정
    if best_score < val_score:
      best_model = model
      best_score = val_score
      es_count = 0

      # checkpoint
      #best_acc_model = deepcopy(model.state_dict())
      print("model save!!")
      torch.save(best_model, '/content/drive/MyDrive/Colab Notebooks/Artist_classification/group_softmax_models/group'+ str(CFG['group_num']) +'.pt')
      
    if es_count > CFG['PATIENCE']:
      print('Early Stopping')
      break

  return best_model

In [21]:
# 이번 대회에서는 F1 score를 사용
def competition_metric(true, pred):
    return f1_score(true, pred, average="macro")

def validation(model, criterion, test_loader, device):

  # 모델을 평가용으로 전환 (dropout 등의 규제가 들어가지 않게 조절)
  model.eval()
  
  model_preds = []
  true_labels = []

  val_loss = []

  # 평가 단계에서 Gradient를 통해 파라미터 값이 업데이트되는 현상을 방지
  with torch.no_grad():
    for img,  himg, label in tqdm(iter(test_loader)):
      img,  himg, label = img.float().to(device), himg.float().to(device), label.to(device)
      model_pred = model(img, himg)
      loss = criterion(model_pred, label)
      val_loss.append(loss.item())

      model_preds += model_pred.argmax(1).detach().cpu().numpy().tolist()
      true_labels += label.detach().cpu().numpy().tolist()

  val_f1 = competition_metric(true_labels, model_preds)
  return np.mean(val_loss), val_f1

In [ ]:
# Run1

model_eff = Network_eff()
model_eff.eval()
optimizer = torch.optim.Adam(params = model_eff.parameters(), lr = CFG["LEARNING_RATE"])

# scheduler
lambda1 = lambda epoch: 0.65 ** epoch
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda1)

#scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2, eta_min=0.01, last_epoch=-1)


infer_model_eff = train(model_eff, optimizer, train_loader, val_loader, scheduler, device)
#infer_model_eff = train(model_eff, optimizer, train_loader, val_loader, None, device)


# g5 = Epoch [7], Train Loss : [0.03825] Val Loss : [1.02316] Val F1 Score : [0.79709]

## Inference

In [122]:
infer_model_eff = torch.load('/content/drive/MyDrive/Colab Notebooks/Artist_classification/group_softmax_models/group' +str(CFG['group_num']) +'.pt')

In [123]:
# TTA (test time augmentation)

tta_transforms = tta.Compose(
    [
        tta.HorizontalFlip(),
        tta.VerticalFlip(), 
    ]
)

tta_model = tta.ClassificationTTAWrapper(infer_model_eff, tta_transforms)


In [124]:
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Artist_classification/data/test.csv')
test.head()

,id,img_path
0,TEST_00000,./test/TEST_00000.jpg
1,TEST_00001,./test/TEST_00001.jpg
2,TEST_00002,./test/TEST_00002.jpg
3,TEST_00003,./test/TEST_00003.jpg
4,TEST_00004,./test/TEST_00004.jpg


In [125]:
test['img_path'] = test['img_path'].apply(lambda x : '/content/drive/MyDrive/Colab Notebooks/Artist_classification/data' + x[1:] )

In [ ]:
test.shape

(12670, 2)

In [126]:
# Test에는 artist 정보가 없으니 infer=True
test_img_paths = get_data(test, infer=True)

In [ ]:
#test_img_paths = ['/content/drive/MyDrive/Colab Notebooks/test.jpg']

In [127]:
test_dataset = TestDataset(test_img_paths, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=2)

In [128]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    
    model_preds = []
    
    with torch.no_grad():
        for img,himg in tqdm(iter(test_loader)):
            img, himg = img.float().to(device), himg.float().to(device)
            
            model_pred = model(img,himg)
            model_preds += model_pred.detach().cpu().numpy().tolist()
    
    print('Done.')
    return model_preds

In [129]:
# eff inference
preds_eff = inference(tta_model, test_loader, device)

  0%|          | 0/198 [00:00<?, ?it/s]

Done.


In [ ]:
pred

array([41])

In [130]:
np.save('/content/drive/MyDrive/Colab Notebooks/Artist_classification/group_softmax_preds/pred_group'+str(CFG['group_num']), preds_eff) 

In [ ]:
pred = np.argmax(preds_eff,axis=1)

In [ ]:
pred.shape

(12670,)

In [ ]:
preds = le.inverse_transform(pred) # LabelEncoder로 변환 된 Label을 다시 화가이름으로 변환

In [ ]:
preds

array(['Edgar Degas', 'Amedeo Modigliani', 'Caravaggio', ...,
       'Amedeo Modigliani', 'Titian', 'Vincent van Gogh'], dtype=object)

## Submit

In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Artist_classification/data/sample_submission.csv')
submit.head()

,id,artist
0,TEST_00000,Edgar Degas
1,TEST_00001,Edgar Degas
2,TEST_00002,Edgar Degas
3,TEST_00003,Edgar Degas
4,TEST_00004,Edgar Degas


In [ ]:
submit['artist'] = preds
submit.head()

,id,artist
0,TEST_00000,Edgar Degas
1,TEST_00001,Amedeo Modigliani
2,TEST_00002,Caravaggio
3,TEST_00003,Albrecht Du rer
4,TEST_00004,Vincent van Gogh


In [ ]:
submit.to_csv('/content/drive/MyDrive/Colab Notebooks/Artist_classification/submission_multimodal_color_hist_20221107.csv', index=False)